### Домашнее задание
Нужно реализовать rest api на базе flask (пример https://github.com/fimochka-sudo/gb_flask_hw_api)

По шагам:
1. выбрать себе датасет (который интересен или нравится больше всего), сделать pipeline (преобразования + модель), сохранить его на диск. Если не хочется пайплайн, то можно без него, но так вам же будет удобнее потом вызывать его из кода сервиса.
2. установить удобную для себя среду разработки (pycharm прекрасен - https://www.jetbrains.com/pycharm/)
3. научиться создавать там виртуальное окружение для python (в правом нижнем углу есть add interpreter)
4. для вашего проекта вам понадобится requirements.txt с пакетами. Можно за основу взять такой файл из проекта выше. Для его установки прям в pycharm можно открыть терминал и сделать pip install -r requirements.txt (находясь в корне проекта конечно же при этом)
5. завести себе аккаунт на guthub (если его еще нет). У самого github есть такой "hello world" по работе с ним - https://guides.github.com/activities/hello-world/
6. научиться запускать gunicorn (установить его конечно). Пример для проекта выше: gunicorn -w 5 -b 127.0.0.1:5000 run_server:app
7. итоговый проект должен содержать: 1) каталог models/ (здесь модель-пайплайн предобученная) 2) файл run_server.py (здесь основной код flask-приложения) 3) requirements.txt (список пакетов, которые у вас используются в проекте) 4) README.md (здесь какое-то описание, что вы делаете, что за данные и т.д)
8. (<b>Опционально</b>): front-end сервис какой-то, который умеет принимать от пользователя введеные данные и ходить в ваш api. На самом деле полезно больше вам, т.к если ваш проект будет далее развиваться (новые модели, интересные подходы), то это хороший пунктик к резюме и в принципе - строчка в портфолио)

### Задача тематического моделирования заявок в чатбот

In [1]:
import numpy as np
import pandas as pd

from nltk.corpus import names
import nltk; nltk.download('stopwords')
# NLTK Stop words
from nltk.corpus import stopwords

import re

from pymorphy2 import MorphAnalyzer

from gensim import corpora, models
from gensim.models import CoherenceModel
from gensim.models import Phrases
from gensim.models.ldamulticore import LdaMulticore
import gensim

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sych_\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, FeatureUnion
import dill

In [3]:
pd.set_option('max_colwidth', 120)
pd.set_option('display.width', 500)

In [4]:
data = pd.read_csv("Заявки в чатбот.csv", encoding = 'utf-16', sep=";")
data.tail(20)

,mesTExt
20136,"Добрый день! В базе Д11 по арт 449499 Свинина охл вес в карточке товара цена SSP и цена продажи не совпадают, ценник..."
20137,"Добрый день. Не работает апс Выключаешь его, включаешь и при включении компьютера он начинает пищать Подключился на ..."
20138,"Добрый день.В автозаказе поставщик Новозеланские продукты не открыта карточка товара код 168181 киви Голд 1шт,Новая ..."
20139,Добрый день Не смогу сменить пароль nan но я ввожу все вернол но я ввожу все вернол ой позвоните еще раз спасибо
20140,Доброго утра: Поставщик Ист Лоджистикал. Подтверждение order все на сегодня nan Спасибо Не осталось
20141,"Добрый день Сегодня была смена юр.лица на самбери 24,при оформление РН,выдал такую ошибку и все документы пустые. na..."
20142,"Доброе утро. \nПрошу перезагрузить RC-LOG59. Спасибо Получилось, спасибо"
20143,"Здравствуйте! ПОдскажите, пожалуйста, как перевести поставщика к EDI? хочет работать так я могу ему направить ДС о п..."
20144,"Да Добрый день.\nНе могу свести заказ, не которые вз не подтягиваются в формирование сводного заказа, например с 01,..."
20145,"Добрый день! Прошу помочь в настройке работы почты на удаленном доступе. Утром подключилась, как обычно, открылись п..."


In [5]:
class TextImputer(BaseEstimator, TransformerMixin):
    def __init__(self, key, value):
        self.key = key
        self.value = value
        
    def get_stopwords(self):
        russian_stopwords = stopwords.words("russian")
        df_sw = pd.read_csv('stopwords.csv', encoding = 'utf-8', sep=";")
        for index, row in df_sw.iterrows():
            russian_stopwords.append(row['stopword'])
        return russian_stopwords
        
    def to_lemmatize2(self, df, key):
        all_word_str = " ".join(df[key])
        all_word_list = all_word_str.split()
        all_unique_word = pd.Series(all_word_list).unique()
        lemmatized_word_dict = {}
        lemmatizer = MorphAnalyzer()
        for word in all_unique_word:
            lemmatized_word_dict[word] = lemmatizer.normal_forms(word)[0]
        lemm_func = lambda text: ' '.join([lemmatized_word_dict[word] for word in text.split()])
        df[key] = df[key].apply(lemm_func)
        return df, all_unique_word
    
    def fit(self, X, y=None):
        return self
    def transform(self, X):

        X[self.key] = X[self.key].replace('—','-')
        
        #1. удаляем пунктуацию
        deleted_symbols = r'[\\\\\'[\]!"$%&()*+,-./:;<=>?№@^_`{|}~«»\n]'  
        func = lambda text : re.sub(deleted_symbols, ' ', str(text))
        X[self.key] = X[self.key].apply(func)
        
        #2. удалим смайлики
        emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
        func = lambda text : re.sub(emoji_pattern, ' ', str(text))
        X[self.key] = X[self.key].apply(func)
        
        #3. удалим отдельно стоящие цифры
        func = lambda text : ' '.join([elem for elem in str(text).split(' ') if elem.isdigit() == False])   
        X[self.key] = X[self.key].apply(func)
        
        #4. приводим к нижнему регистру
        X[self.key] = X[self.key].apply(lambda text : text.lower())
        
        #5. лемматизация (приводим слова к начальной форме)
        X, _ = self.to_lemmatize2(X, self.key)
        
        #6. удаляем стоп слова
        sw = self.get_stopwords()
        func = lambda text : ' '.join([elem for elem in str(text).split(' ') if elem not in sw and not elem in ['nan', np.nan]])   
        X[self.key] = X[self.key].apply(func)
        
        return X 
    
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        #приведем к виду списка списков, потом этот список списков пойдет в модель LDA
        lst = X[self.key].to_list()
        texts = []
        for i in range(len(lst)):    
            texts.append(lst[i].split(' '))
        return texts
    
class myLDA():
    def __init__(self, num_topics, passes, alpha, eta):
        
        self.num_topics=num_topics 
        self.passes = passes 
        self.alpha = alpha 
        self.eta = eta 
    
    def fit(self, texts):
        
        text_clean = texts.copy()
        bigram = Phrases(text_clean) # Создаем биграммы на основе корпуса
        trigram = Phrases(bigram[text_clean])# Создаем триграммы на основе корпуса
        
        for idx in range(len(text_clean)):
            for token in bigram[text_clean[idx]]:
                if '_' in token:
                    # Токен это би грамма, добавим в документ.
                    text_clean[idx].append(token)
            for token in trigram[texts[idx]]:
                if '_' in token:
                    # Токен это три грамма, добавим в документ.
                    text_clean[idx].append(token)
        
        dictionary_bt = corpora.Dictionary(text_clean)   # составляем словарь
        dictionary_bt.filter_extremes(no_below=10, no_above=0.1)
        corpus_bt = [dictionary_bt.doc2bow(text) for text in text_clean]  # составляем корпус документов
        model = LdaMulticore(corpus = corpus_bt, 
                             id2word = dictionary_bt, 
                             num_topics = self.num_topics, 
                             passes = self.passes, 
                             alpha = self.alpha, 
                             eta = self.eta)
        return model
    
    def predict(self, text):
        #clean_text = self.clean(text)
        #bigram = self.bigram([clean_text])
        #new_review_bow = self.dictionary.doc2bow(bigram[0])
        #new_review_lda = self.lda[new_review_bow]
        #return sorted(new_review_lda, reverse=True, key=itemgetter(1))
        return 0

In [6]:
#example
description = Pipeline([
                ('imputer', TextImputer('mesTExt', '')),
                ('selector', ColumnSelector(key='mesTExt'))
            ])

#description.fit(data)
#description.transform(data)

In [7]:
data.head(3)

,mesTExt
0,124 1 Привет
1,Привет Привет Привет Пока 1 End Привет 1 11 END ПРивет У меня сломался ноготь У меня сломан ноготь У меня сломан ног...
2,Привет Хабаровск 11 У меня сломался ноутбук Не рабртает автозаказ Не работает автозаказ


In [8]:
pipeline = Pipeline([
    ('description',description),
    ('lda', myLDA(num_topics=16, passes=20, alpha=1.25, eta=1.25)),
])

model = pipeline.fit(data)


TypeError: fit() takes 2 positional arguments but 3 were given